# From relational to graph databases

In [ ]:
# Uncomment to install packages
# !pip install -r requirements.txt

## MySQL settings

In [2]:
PASSWORD = '<your_MYSQL_instance_password>'
import mysql.connector

connection = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd=PASSWORD,
  database="steam_data")

In [3]:
# Uses the MySQL function we created in the previous step
from graphtastic.database.mysql import query_mysql

## Schema design

### Ingestion considerations

In [4]:
play_query = 'SELECT id, game_name, hours FROM steam_play'
play_data = query_mysql(play_query, password=PASSWORD)
print(play_data[:10])

[('151603712', 'The Elder Scrolls V Skyrim', 273.0), ('151603712', 'Fallout 4', 87.0), ('151603712', 'Spore', 14.9), ('151603712', 'Fallout New Vegas', 12.1), ('151603712', 'Left 4 Dead 2', 8.9), ('151603712', 'HuniePop', 8.5), ('151603712', 'Path of Exile', 8.1), ('151603712', 'Poly Bridge', 7.5), ('151603712', 'Left 4 Dead', 3.3), ('151603712', 'Team Fortress 2', 2.8)]


In [5]:
purchase_query = 'SELECT id, game_name FROM steam_purchase'
purchase_data = query_mysql(purchase_query,password=PASSWORD )
print(purchase_data[:10])

[('151603712', 'The Elder Scrolls V Skyrim'), ('151603712', 'Fallout 4'), ('151603712', 'Spore'), ('151603712', 'Fallout New Vegas'), ('151603712', 'Left 4 Dead 2'), ('151603712', 'HuniePop'), ('151603712', 'Path of Exile'), ('151603712', 'Poly Bridge'), ('151603712', 'Left 4 Dead'), ('151603712', 'Team Fortress 2')]


In [6]:
users = set([row[0] for row in play_data] + [row[0] for row in purchase_data])
user_ids = {user_id: igraph_id for igraph_id, user_id in enumerate(users)}
print(len(user_ids))

12393


In [7]:
games = set([row[1] for row in play_data] + [row[1] for row in purchase_data])
game_ids = {user_id: igraph_id for igraph_id, user_id in enumerate(games, len(user_ids))}
print(len(game_ids))

5155


In [8]:
print(sorted(user_ids.values(), reverse=True)[:10])
print(sorted(game_ids.values(), reverse=False)[:10])

[12392, 12391, 12390, 12389, 12388, 12387, 12386, 12385, 12384, 12383]
[12393, 12394, 12395, 12396, 12397, 12398, 12399, 12400, 12401, 12402]


In [9]:
all_ids = sorted(list(user_ids.values()) + list(game_ids.values()))
assert all_ids == list(range(len(all_ids)))

In [10]:
import igraph as ig
g = ig.Graph(directed=True)

In [11]:
user_ids = dict(sorted(user_ids.items(), key=lambda item: item[1]))
game_ids = dict(sorted(game_ids.items(), key=lambda item: item[1]))

In [12]:
steam_user_ids = list(user_ids.keys())
steam_game_ids = list(game_ids.keys())

In [13]:
g.add_vertices(len(steam_user_ids) + len(steam_game_ids))
assert len(g.vs) == len(steam_user_ids) + len(steam_game_ids)

In [14]:
all_steam_ids = steam_user_ids + steam_game_ids

In [15]:
node_types = ['user' for _ in steam_user_ids] + ['game' for _ in steam_game_ids]

In [16]:
g.vs['steam_id'] = all_steam_ids
g.vs['type'] = node_types

In [17]:
print(g.vs['steam_id'][:10])
print(g.vs['type'][:10])

['119906029', '196724692', '63024728', '40601456', '192080216', '239138885', '27152433', '162580876', '195611290', '94161846']
['user', 'user', 'user', 'user', 'user', 'user', 'user', 'user', 'user', 'user']


In [18]:
game_nodes = g.vs.select(type_eq='game')
print(len(game_nodes))

5155


In [19]:
purchase_edges = [[user_ids[user], game_ids[purchase]]
               	for user, purchase in purchase_data]
play_edges = [[user_ids[user], game_ids[game], hours]
           	for user, game, hours in play_data]

In [20]:
g.add_edges([(n, m) for n, m, _ in play_edges])
g.es['hours'] = [hours for _, _, hours in play_edges]

In [21]:
g.add_edges(purchase_edges)

In [22]:
edge_type = ['PLAYED' for _ in play_edges] + ['PURCHASED' for _ in purchase_edges]
g.es['edge_type'] = edge_type

In [23]:
user_id_ex = g.vs.select(steam_id_eq='151603712')[0].index
purchased_ex = g.es.select(_source_eq=user_id_ex, edge_type='PURCHASED')
print(len(list(purchased_ex)))

40


## Path based analytics in igraph

In [24]:
paths = g.get_all_simple_paths(user_id_ex, cutoff=3, mode='all')
print(paths[:10])

[[9686, 12421], [9686, 12421, 9], [9686, 12421, 9, 12474], [9686, 12421, 9, 12744], [9686, 12421, 9, 12793], [9686, 12421, 9, 13116], [9686, 12421, 9, 13475], [9686, 12421, 9, 13963], [9686, 12421, 9, 14019], [9686, 12421, 9, 14072]]


In [25]:
rec_game_ids = [path[3] for path in paths if len(path) == 4]

In [26]:
game_names = [g.vs[game_id]['steam_id'] for game_id in rec_game_ids]

In [27]:
neighbors = g.neighbors(user_id_ex)
purchased_games = [g.vs[node_id]['steam_id'] for node_id in g.neighbors(user_id_ex)]

In [28]:
game_names = [game for game in game_names if game not in purchased_games]

In [29]:
from collections import Counter
game_frequency = Counter(game_names)
print(game_frequency)

Counter({'Counter-Strike Global Offensive': 5421, 'Unturned': 4155, 'Portal 2': 3998, 'Half-Life 2 Lost Coast': 3737, 'Portal': 3563, 'Borderlands 2': 3556, 'Counter-Strike Source': 3530, 'Terraria': 3455, 'Half-Life 2': 3398, "Sid Meier's Civilization V": 3085, 'Warframe': 3070, 'Half-Life 2 Deathmatch': 2805, 'PAYDAY 2': 2791, 'Metro 2033': 2668, 'Half-Life 2 Episode Two': 2593, 'Half-Life 2 Episode One': 2590, 'The Witcher 2 Assassins of Kings Enhanced Edition': 2493, 'Saints Row The Third': 2479, 'Skyrim High Resolution Texture Pack': 2343, 'Counter-Strike': 2269, 'Just Cause 2': 2202, 'War Thunder': 2173, 'Batman Arkham City GOTY': 2152, 'Heroes & Generals': 2102, 'Magicka': 2101, 'Dishonored': 2018, 'Torchlight II': 1967, 'PAYDAY The Heist': 1934, 'PlanetSide 2': 1927, 'Patch testing for Chivalry': 1910, 'Chivalry Medieval Warfare': 1910, "Mirror's Edge": 1897, 'Killing Floor': 1895, 'Dead Island': 1894, 'Amnesia The Dark Descent': 1880, 'Killing Floor Mod Defence Alliance 2': 18